In [18]:
import torch
from Scaled_KAN import FastKAN
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_conv = FastKAN(
                layers_hidden=[784,1569,10],
                num_grids=4,
                device=device
            )
model_continual_ewc = FastKAN(
                layers_hidden=[784,1569,10],
                num_grids=4,
                device=device
            )
model_continual_replay = FastKAN(
                layers_hidden=[784,1569,10],
                num_grids=4,
                device=device
            )

model_conv.load_state_dict(torch.load('FastKAN_4_6M.pth'))
model_conv.eval()

model_continual_ewc.load_state_dict(torch.load('continual_mnist_FastKAN_1569_ewc.pth'))
model_continual_ewc.eval()

model_continual_replay.load_state_dict(torch.load('continual_mnist_FastKAN_1569_replay.pth'))
model_continual_replay.eval()

layers_conv = []
layers_continual_ewc = []
layers_continual_replay = []
for layer in model_conv.modules():
    layers_conv.append(layer)
for layer in model_continual_ewc.modules():
    layers_continual_ewc.append(layer)
for layer in model_continual_replay.modules():
    layers_continual_replay.append(layer)

layer_conv = layers_conv[1]
layer_continual_ewc = layers_continual_ewc[1]
layer_continual_replay = layers_continual_replay[1]


# Assuming layer[1].plot_curve is a function that takes two arguments and returns x, y coordinates
def plot_curve_with_slider(j):
    x_conv, y_conv = layer_conv[1].plot_curve(0, j, num_pts=100, num_extrapolate_bins=3)
    x_continual_ewc, y_continual_ewc = layer_continual_ewc[1].plot_curve(0, j, num_pts=100, num_extrapolate_bins=3)
    x_continual_replay, y_continual_replay = layer_continual_replay[1].plot_curve(0, j, num_pts=100, num_extrapolate_bins=3)
    x_conv, y_conv = x_conv.cpu().numpy(), y_conv.cpu().numpy()
    x_continual_ewc, y_continual_ewc = x_continual_ewc.cpu().numpy(), y_continual_ewc.cpu().numpy()
    x_continual_replay, y_continual_replay = x_continual_replay.cpu().numpy(), y_continual_replay.cpu().numpy()
    plt.plot(x_conv, y_conv, label='Conv')
    plt.plot(x_continual_ewc, y_continual_ewc, label='Continual EWC')
    plt.plot(x_continual_replay, y_continual_replay, label='Continual Replay')
    plt.legend()
    plt.show()
# Create an interactive slider
interactive_plot = interactive(plot_curve_with_slider, j=widgets.SelectionSlider(options=[i for i in range(10)]))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(SelectionSlider(description='j', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Outpu…